In [6]:
#!/usr/bin/env python
# coding: utf-8

import os
import math
import pandas as pd
import numpy as np
from pathlib import Path
import shutil
import argparse
from typing import List, Union
import matplotlib.pyplot as plt
import yaml
from datetime import datetime
import logging
import time

import torch
from torch import Tensor, nn
from torch.types import Device, _size
from torch.nn.parameter import Parameter, UninitializedParameter
from torch.nn import init
from torch.utils.data import Dataset
from torch.utils.data import ConcatDataset
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from collections import OrderedDict
from torch.nn import functional as F

from configs.config import configs

# from models.pe import PositionalEncoding
from EEG_ARNN.gcnModelST_pytorch import GCN_layer
from EEG_ARNN.nnModelST_pytorch import zhnn
from models.encoder import res_encoderS, res_encoderM

from models.classifier import transformer_classifier



def preprocess_adj(adj):
    adj = adj + np.eye(adj.shape[0])
    adj = normalize_adj(adj)
    return adj

def normalize_adj(adj):
    d = np.diag(np.power(np.array(adj.sum(1)), -0.5).flatten(), 0)
    a_norm = adj.dot(d).transpose().dot(d)
    return a_norm

# Transform signal
def transform(data:Tensor, mean:Tensor, std:Tensor):
    normalized_data = (data - mean) / std
    return normalized_data

In [3]:
df = pd.read_excel('./configs/init_adj_tuh.xlsx')
Abf = df.iloc[:, 1:].values
A = preprocess_adj(Abf)
# A = np.ones((60,60))
A = np.float32(A)
A = torch.from_numpy(A)

In [4]:
classifier = zhnn((19,12000), A)
data = torch.rand(4,19,12000)

In [9]:
y  = classifier(data)

RuntimeError: Given groups=1, weight of size [16, 1, 1, 16], expected input[1, 4, 19, 12015] to have 1 channels, but got 4 channels instead

In [7]:
x = F.pad(data, pad=(7, 8, 0, 0))

In [8]:
x.shape

torch.Size([4, 19, 12015])